## Load FRETBursts software

In [1]:
from fretbursts import *

 - Optimized (cython) burst search loaded.
 - Optimized (cython) photon counting loaded.
--------------------------------------------------------------
 You are running FRETBursts (version 0.5.9).

 If you use this software please cite the following paper:

   FRETBursts: An Open Source Toolkit for Analysis of Freely-Diffusing Single-Molecule FRET
   Ingargiola et al. (2016). http://dx.doi.org/10.1371/journal.pone.0160716 

--------------------------------------------------------------


In [2]:
import phconvert as phc
phc.__version__

'0.7.2'

In [3]:
def smfret_multispot_ni_32_32(
        filename,
        excitation_wavelengths=(532e-9,),
        detection_wavelengths = (580e-9,),
        software = 'LabVIEW Multi-channel counter',
        swap_D_A = True,
        setup = None):
    """Load multi-spot smFRET files (ni64) and returns a dictionary.

    This dictionary can be passed to the :func:`phconvert.hdf5.photon_hdf5`
    function to save the data in Photon-HDF5 format.
    """
    ts_unit = 12.5e-9
    #print(" - Loading '%s' ... " % filename)
    timestamps_m, A_em, _ = loader.load_data_ordered16(fname=filename, swap_D_A=swap_D_A)
    #print(" [DONE]\n")

    if setup is None:
        setup = dict(
            num_pixels = 16,
            num_spots = 8,
            num_spectral_ch = 2,
            num_polarization_ch = 1,
            num_split_ch = 1,
            modulated_excitation = False,
            lifetime = False,
            excitation_wavelengths = excitation_wavelengths,
            excitation_cw = (False,),
            detection_wavelengths = detection_wavelengths)

    provenance = dict(filename=filename, software=software)
    acquisition_duration = np.max([t[-1] - t[0] for t in timestamps_m])*ts_unit
    data = dict(
        _filename = filename,
        acquisition_duration = round(acquisition_duration),
        provenance=provenance)
    
    if setup != 'skip':
        data['setup'] = setup

    for ich, (times, a_em) in enumerate(zip(timestamps_m, A_em)):
        data.update(
            {'photon_data%d' % ich:
             dict(
                timestamps = times,
                timestamps_specs = dict(timestamps_unit=ts_unit),
                detectors = a_em.view('uint8'),

                measurement_specs = dict(
                    measurement_type = 'smFRET',
                    detectors_specs = dict(spectral_ch1 = 0,
                                           spectral_ch2 = 1)))}
            )

    return data

## Data files

In [4]:
data_dir = r'C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/'
#data_dir = r'data/'

f7 = '7d_New_150p_320mW_steer_3.dat'
f12 = '12d_New_30p_320mW_steer_3.dat'
f17 = '17d_100p_320mW_steer_1.dat'
f22 = '22d_30p_320mW_steer_1.dat'
f27 = '27d_50p_320mW_steer_1.dat'
fo = 'DO12_No2_50p_320mW_steer_1.dat'
fb = 'TE50_320mW_3.dat'

f7, f12, f17, f22, f27, fo, fb = [data_dir+f for f in [f7, f12, f17, f22, f27, fo, fb]]
f_dict = {'7d': f7, '12d': f12, '17d': f17, '22d': f22, '27d': f27, 'do': fo, 'buffer': fb}

# Author

In [5]:
author = 'Antonino Ingargiola'
author_affiliation = 'UCLA'
creator = 'Antonino Ingargiola'
creator_affiliation = 'UCLA'

# Sample

In [6]:
description = '8-spot smFRET measurement of dsDNA.'
sample_name = 'Doubly-labeled 40-bp dsDNA with D-A separation of %d bases.'
sample_name_do = '40-bp dsDNA labeled with a single donor dye.'
dye_names = 'ATTO550, ATTO647N'
dye_names_do = 'ATTO550'
buffer_name = 'TE50'

# Convert one file

In [7]:
fname = f12
meas_id = 12

In [8]:
data = smfret_multispot_ni_32_32(fname)

data['description'] = description

data['sample'] = dict(
    sample_name=sample_name % meas_id,
    dye_names=dye_names,
    buffer_name=buffer_name,
    num_dyes = len(dye_names.split(',')))

data['identity'] = dict(
    author=author,
    author_affiliation=author_affiliation,
    creator=creator,
    creator_affiliation=creator_affiliation)

 - Loading data "C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/12d_New_30p_320mW_steer_3.dat" ...  [DONE]
 - Processing data ... 
   - Swapping D and A channels ...  [DONE]
   [DONE Processing]


In [9]:
phc.hdf5.save_photon_hdf5(data, overwrite=True)

Saving: C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/12d_New_30p_320mW_steer_3.hdf5


> *Stop when converting only one file*


# Batch conversion

In [10]:
for name, fname in f_dict.items():
    if name == 'buffer':
        continue
    print (name, fname)
    
    data = smfret_multispot_ni_32_32(fname)

    data['description'] = description

    data['identity'] = dict(
        author=author,
        author_affiliation=author_affiliation,
        creator=creator,
        creator_affiliation=creator_affiliation)

    if name == 'do':
        data['sample'] = dict(
            sample_name=sample_name_do,
            dye_names=dye_names_do,
            buffer_name=buffer_name,
            num_dyes = len(dye_names_do.split(',')))

    else:
        data['sample'] = dict(
            sample_name=sample_name % int(name[:-1]),
            dye_names=dye_names,
            buffer_name=buffer_name,
            num_dyes = len(dye_names.split(',')))
        
    phc.hdf5.save_photon_hdf5(data, overwrite=True)

7d C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/7d_New_150p_320mW_steer_3.dat
 - Loading data "C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/7d_New_150p_320mW_steer_3.dat" ...  [DONE]
 - Processing data ... 
   - Swapping D and A channels ...  [DONE]
   [DONE Processing]
Saving: C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/7d_New_150p_320mW_steer_3.hdf5
22d C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/22d_30p_320mW_steer_1.dat
 - Loading data "C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/22d_30p_320mW_steer_1.dat" ...  [DONE]
 - Processing data ... 
   - Swapping D and A channels ...  [DONE]
   [DONE Processing]
Saving: C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/22d_30p_320mW_steer_1.hdf5
17d C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/17d_100p_320mW_steer_1.dat
 - Loading data "C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/17d_100p_320mW_steer_1.dat" ...  [DONE]
 - Processing data ... 
   - Swapping D and A channels ... 

# DCR files

In [11]:
filename = r'C:\Data\Antonio\data\8-spot 5samples data\2014-02-26\DCR_test_2.dat'

In [12]:
setup = dict(
    num_pixels = 16,
    )

In [13]:
data = smfret_multispot_ni_32_32(filename, setup='skip')

data['description'] = 'DCR measurement for two 8-pixel POLIMI SPAD arrays (octopus).'

data['identity'] = dict(
    author=author,
    author_affiliation=author_affiliation,
    creator=creator,
    creator_affiliation=creator_affiliation)

 - Loading data "C:\Data\Antonio\data\8-spot 5samples data\2014-02-26\DCR_test_2.dat" ...  [DONE]
 - Processing data ... 
   - Swapping D and A channels ...  [DONE]
   [DONE Processing]


In [14]:
phc.hdf5.save_photon_hdf5(data, overwrite=True, require_setup=False)

Saving: C:\Data\Antonio\data\8-spot 5samples data\2014-02-26\DCR_test_2.hdf5


In [15]:
data['_data_file'].close()

In [16]:
phc.hdf5.load_photon_hdf5(r'C:\Data\Antonio\data\8-spot 5samples data\2014-02-26\DCR_test_2.hdf5', require_setup=False)

/ (RootGroup) b'A file format for photon-counting detector based single-molecule spectroscopy experiments.'
  children := ['photon_data2' (Group), 'photon_data3' (Group), 'photon_data1' (Group), 'photon_data5' (Group), 'photon_data7' (Group), 'photon_data6' (Group), 'photon_data4' (Group), 'photon_data0' (Group), 'provenance' (Group), 'acquisition_duration' (Array), 'identity' (Group), 'description' (Array)]

# Buffer file

In [17]:
description = '8-spot smFRET measurement of TE50 buffer.'
sample_name = 'Buffer TE50 only.'
dye_names = ''
buffer_name = 'TE50'

In [18]:
d = loader.multispot8_core(fname=fb)
d

 - Loading data "C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/TE50_320mW_3.dat" ...  [DONE]
 - Processing data ... 
   - Swapping D and A channels ...  [DONE]
   [DONE Processing]


2013-05-15_TE50_320mW_3 G1.000

In [19]:
fname = fb

In [20]:
data = smfret_multispot_ni_32_32(fname)

data['description'] = description

data['sample'] = dict(
    sample_name=sample_name,
    buffer_name=buffer_name,
    num_dyes = len(dye_names.split(',')))

data['identity'] = dict(
    author=author,
    author_affiliation=author_affiliation,
    creator=creator,
    creator_affiliation=creator_affiliation)

 - Loading data "C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/TE50_320mW_3.dat" ...  [DONE]
 - Processing data ... 
   - Swapping D and A channels ...  [DONE]
   [DONE Processing]


In [21]:
phc.hdf5.save_photon_hdf5(data, overwrite=True)

Saving: C:/Data/Antonio/data/8-spot 5samples data/2013-05-15/TE50_320mW_3.hdf5
